# Batch Processing Example

In this example, we use the `micasense.imageset` class to load a set of directories of images into a list of `micasense.capture` objects, and we iterate over that list saving out each image as an aligned stack of images as separate bands in a single tiff file each. Next, we use the metadata from the original captures to write out a log file of the captures and their locations.  Finally, we use `exiftool` from the command line to inject that metadata into the processed images, allowing us to stitch those images using commercial software such as Pix4D or Agisoft.

Note: for this example to work, the images must have a valid RigRelatives tag. This requires RedEdge version of at least 3.4.0 or any version of Altum.  If your images don't meet that spec, you can also follow this support ticket to add the RigRelatives tag to them: https://support.micasense.com/hc/en-us/articles/360006368574-Modifying-older-collections-for-Pix4Dfields-support

In [ ]:
%load_ext autoreload
%autoreload 2

## Load Images into ImageSet

In [ ]:
from ipywidgets import FloatProgress, Layout
from IPython.display import display
import micasense.imageset as imageset
import micasense.capture
import os, glob
import multiprocessing

images_dir = os.path.expanduser(os.path.join('~','Downloads','RedEdgeImageSet','0000SET'))
panelNames = glob.glob(os.path.join(images_dir,'000','IMG_0000_*.tif'))
panelCap = micasense.capture.Capture.from_filelist(panelNames)

# Make sure to update or replace this if using your own data.
# Setting panel_irradiance=None should use built-in panel calibration if present
panel_reflectance_by_band = [0.67, 0.69, 0.68, 0.61, 0.67] #RedEdge band_index order
panel_irradiance = panelCap.panel_irradiance(panel_reflectance_by_band)

if panelCap.panels_in_all_expected_images():
    panel_irradiance = panelCap.panel_irradiance(panel_irradiance)
else:
    raise ValueError("Panels not detected in all images")



In [ ]:
## This progress widget is used for display of the long-running process
f = FloatProgress(min=0, max=1, step=0.01, layout=Layout(width='100%'), description="Loading")
display(f)
def update_f(val):
    f.value=val

%time imgset = imageset.ImageSet.from_directory(images_dir, progress_callback=update_f)
update_f(1.0)

## Define which warp method to use
For newer data sets with RigRelatives tags (images captured with RedEdge version 3.4.0 or greater with a valid calibration load, see https://support.micasense.com/hc/en-us/articles/360005428953-Updating-RedEdge-for-Pix4Dfields), we can use the RigRelatives for a simple alignment.

For sets without those tags, or sets that require a RigRelatives optimization, we can go through the Alignment.ipynb notebook and get a set of `warp_matrices` that we can use here to align.

In [ ]:
warp_matrices = None # Set warp_matrices to none to align using RigRelatives

from numpy import array
from numpy import float32

# Use the warp_matrices derived from the Alignment Tutorial for this RedEdge set without RigRelatives
warp_matrices = [array([[ 1.0022864e+00, -2.5218742e-03, -7.8897724e+00],
       [ 2.3614613e-03,  1.0036647e+00, -1.3134360e+01],
       [-1.7785960e-06,  1.1342933e-06,  1.0000000e+00]], dtype=float32), array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32), array([[ 9.9724644e-01, -1.5535479e-03,  1.2301332e+00],
       [ 8.6746749e-04,  9.9738181e-01, -1.6499146e+00],
       [-8.2814904e-07, -3.4492047e-07,  1.0000000e+00]], dtype=float32), array([[ 1.0007139e+00, -8.4427306e-03,  1.6312788e+01],
       [ 6.2834658e-03,  9.9977130e-01, -1.6011746e+00],
       [-1.9520005e-06, -6.3761996e-07,  1.0000000e+00]], dtype=float32), array([[ 9.9284184e-01,  9.2153507e-04,  1.6069824e+01],
       [-3.2895375e-03,  9.9262553e-01, -5.0334185e-01],
       [-1.5845230e-06, -1.7681158e-06,  1.0000000e+00]], dtype=float32)]

## Align images and save each capture to a layered tiff file

In [ ]:
import exiftool
import datetime
## This progress widget is used for display of the long-running process
f2 = FloatProgress(min=0, max=1, step=0.01, layout=Layout(width='100%'), description="Saving")
display(f2)
def update_f2(val):
    f2.value=val

outputPath = os.path.join(imgset.basedir,'..','stacks')
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

overwrite = False
try:
    irradiance = panel_irradiance+[0]
except NameError:
    irradiance = None

start = datetime.datetime.now()
for i,capture in enumerate(imgset.captures):
    outputFilename = capture.uuid+'.tif'
    fullOutputPath = os.path.join(outputPath, outputFilename)
    if (not os.path.exists(fullOutputPath)) or overwrite:
        capture.save_capture_as_reflectance_stack(fullOutputPath, irradiance_list=irradiance, warp_matrices=warp_matrices)
    capture.clear_image_data()
    update_f2(float(i)/float(len(imgset.captures)))
update_f2(1.0)
end = datetime.datetime.now()

print("Saving time: {}".format(end-start))
print("Alignment+Saving rate: {:.2f} images per second".format(float(len(imgset.captures))/float((end-start).total_seconds())))

## Extract Metadata from Captures list and save to log.csv

In [ ]:
def decdeg2dms(dd):
   is_positive = dd >= 0
   dd = abs(dd)
   minutes,seconds = divmod(dd*3600,60)
   degrees,minutes = divmod(minutes,60)
   degrees = degrees if is_positive else -degrees
   return (degrees,minutes,seconds)

header = "SourceFile,\
GPSDateStamp,GPSTimeStamp,\
GPSLatitude,GpsLatitudeRef,\
GPSLongitude,GPSLongitudeRef,\
GPSAltitude,GPSAltitudeRef,\
FocalLength,\
XResolution,YResolution,ResolutionUnits\n"

lines = [header]
for capture in imgset.captures:
    #get lat,lon,alt,time
    outputFilename = capture.uuid+'.tif'
    fullOutputPath = os.path.join(outputPath, outputFilename)
    lat,lon,alt = capture.location()
    #write to csv in format:
    # IMG_0199_1.tif,"33 deg 32' 9.73"" N","111 deg 51' 1.41"" W",526 m Above Sea Level
    latdeg, latmin, latsec = decdeg2dms(lat)
    londeg, lonmin, lonsec = decdeg2dms(lon)
    latdir = 'North'
    if latdeg < 0:
        latdeg = -latdeg
        latdir = 'South'
    londir = 'East'
    if londeg < 0:
        londeg = -londeg
        londir = 'West'
    resolution = capture.images[0].focal_plane_resolution_px_per_mm

    linestr = '"{}",'.format(fullOutputPath)
    linestr += capture.utc_time().strftime("%Y:%m:%d,%H:%M:%S,")
    linestr += '"{:d} deg {:d}\' {:.2f}"" {}",{},'.format(int(latdeg),int(latmin),latsec,latdir[0],latdir)
    linestr += '"{:d} deg {:d}\' {:.2f}"" {}",{},{:.1f} m Above Sea Level,Above Sea Level,'.format(int(londeg),int(lonmin),lonsec,londir[0],londir,alt)
    linestr += '{}'.format(capture.images[0].focal_length)
    linestr += '{},{},mm'.format(resolution,resolution)
    linestr += '\n' # when writing in text mode, the write command will convert to os.linesep
    lines.append(linestr)

fullCsvPath = os.path.join(outputPath,'log.csv')
with open(fullCsvPath, 'w') as csvfile: #create CSV
    csvfile.writelines(lines)

## Use Exiftool from the command line to write metadata to images

In [ ]:
import subprocess

old_dir = os.getcwd()
os.chdir(outputPath)
cmd = 'exiftool -csv="{}" -overwrite_original .'.format(fullCsvPath)
print(cmd)
try:
    subprocess.check_call(cmd)
finally:
    os.chdir(old_dir)